In [1]:
# Transfer Learning : ResNet50

In [2]:
# required imports

from tensorflow.keras.layers import  Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

In [3]:
# loading resnet50 architecture
res_net_50 = ResNet50(input_shape=[224, 224] + [3], weights='imagenet', include_top=False)

# acquiring trained parameters for the model
for layer in res_net_50.layers:
    layer.trainable = False
  

In [4]:
# adding a flatten layer
modified_res_net_50 = Flatten()(res_net_50.output)

# adding an output layer
prediction = Dense(211, activation='softmax')(modified_res_net_50)

# create a model object
modified_res_net_50 = Model(inputs=res_net_50.input, outputs=prediction)

# view the structure of the model
modified_res_net_50.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [5]:
# compile the model

modified_res_net_50.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [6]:
# required imports

from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [7]:
# using custom variations as per the resnet50 model
data_gen_resnet50 = ImageDataGenerator(preprocessing_function=preprocess_input)

In [8]:
# extract training images from directory

training_set = data_gen_resnet50.flow_from_directory(
                directory = "Currency_Images/coins/data/train",
                class_mode = 'categorical',
                target_size = (224, 224),
                batch_size = 32,
                subset = 'training'
                )

Found 6413 images belonging to 211 classes.


In [9]:
# validation set images extracted from directory

validation_set = data_gen_resnet50.flow_from_directory(
        directory = 'Currency_Images/coins/data/validation',
        class_mode='categorical',
        target_size = (224, 224)
)

Found 844 images belonging to 211 classes.


In [10]:
# test images extracted from directory

test_set = data_gen_resnet50.flow_from_directory(
        directory = 'Currency_Images/coins/data/test',
        class_mode='categorical',
        target_size = (224, 224),
        batch_size=32
)

Found 844 images belonging to 211 classes.


In [11]:
# model fit on the training set

history_resnet50 = modified_res_net_50.fit_generator(
                                        training_set,
                                        validation_data = validation_set,
                                        epochs = 10
                             )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
201/201 [==============================] - 1072s 5s/step - loss: 23.2135 - accuracy: 0.5257 - val_loss: 17.4045 - val_accuracy: 0.6505
Epoch 2/10
201/201 [==============================] - 1185s 6s/step - loss: 4.4656 - accuracy: 0.8726 - val_loss: 18.6179 - val_accuracy: 0.6991
Epoch 3/10
201/201 [==============================] - 1207s 6s/step - loss: 3.9412 - accuracy: 0.9181 - val_loss: 18.5395 - val_accuracy: 0.7275
Epoch 4/10
201/201 [==============================] - 1142s 6s/step - loss: 3.7892 - accuracy: 0.9205 - val_loss: 22.5091 - val_accuracy: 0.7121
Epoch 5/10
201/201 [==============================] - 904s 4s/step - loss: 3.6942 - accuracy: 0.9281 - val_loss: 24.8630 - val_accuracy: 0.7062
Epoch 6/10
201/201 [==============================] - 934s 5s/step - loss: 3.5733 - accuracy: 0.9457 - val_loss: 24.6402 - val_accuracy: 0.7204
Epoch 7/10
201/201 [==============================] - 9

In [12]:
# final evaluation on the test set

score_resnet50 = modified_res_net_50.evaluate_generator(test_set, verbose=2)


Instructions for updating:
Please use Model.evaluate, which supports generators.
27/27 - 73s - loss: 25.0446 - accuracy: 0.7583


In [13]:
# test loss and test accuracy

print('Resnet50 Model : Test Loss is : ', score_resnet50[0])
print('Resnet50 Model : Test Accuracy is : ', score_resnet50[1])

Resnet50 Model : Test Loss is :  25.044647216796875
Resnet50 Model : Test Accuracy is :  0.758293867111206


In [14]:
modified_res_net_50.save('Vgg16_model_for_currency_classification.h5')